# 1. Convolutional Neural Networks

In [5]:
import torch.nn as nn
import pandas as pd
import torch
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader

## 1.1 Loading and Preparing the Data

### 1.1.1 Preparing Images

In [14]:
train_df = pd.read_csv("data/asl_data/sign_mnist_train.csv")
valid_df = pd.read_csv("data/asl_data/sign_mnist_valid.csv")

This ASL data is already flattened.

In [15]:
sample_df = train_df.head().copy()  # Grab the top 5 rows
sample_df.pop('label')
sample_x = sample_df.values
sample_x

array([[107, 118, 127, ..., 204, 203, 202],
       [155, 157, 156, ..., 103, 135, 149],
       [187, 188, 188, ..., 195, 194, 195],
       [211, 211, 212, ..., 222, 229, 163],
       [164, 167, 170, ..., 163, 164, 179]], shape=(5, 784))

In [16]:
sample_x.shape

(5, 784)

In [17]:
IMG_HEIGHT = 28
IMG_WIDTH = 28
IMG_CHS = 1

sample_x = sample_x.reshape(-1, IMG_CHS, IMG_HEIGHT, IMG_WIDTH)
sample_x.shape

(5, 1, 28, 28)

### 1.1.2 Create a Dataset

In [23]:
class MyDataset(Dataset):
    def __init__(self, base_df):
        x_df = base_df.copy() 
        y_df = x_df.pop('label')
        x_df = x_df.values / 255  # Normalize values from 0 to 1
        x_df = x_df.reshape(-1, IMG_CHS, IMG_WIDTH, IMG_HEIGHT)
        self.xs = torch.tensor(x_df).float()
        self.ys = torch.tensor(y_df)

    def __getitem__(self, idx):
        x = self.xs[idx]
        y = self.ys[idx]
        return x, y

    def __len__(self):
        return len(self.xs)

### 1.1.3 Create a DataLoader

In [24]:
BATCH_SIZE = 32

train_data = MyDataset(train_df)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE)
train_N = len(train_loader.dataset)

valid_data = MyDataset(valid_df)
valid_loader = DataLoader(valid_data, batch_size=BATCH_SIZE)
valid_N = len(valid_loader.dataset)

In [25]:
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

In [26]:
batch = next(iter(train_loader))
batch

[tensor([[[[0.6824, 0.6902, 0.6980,  ..., 0.6706, 0.6667, 0.6549],
           [0.6863, 0.6980, 0.7059,  ..., 0.6745, 0.6706, 0.6627],
           [0.6902, 0.7020, 0.7137,  ..., 0.6863, 0.6745, 0.6706],
           ...,
           [0.7961, 0.8039, 0.8392,  ..., 0.7922, 0.7843, 0.7804],
           [0.7843, 0.8078, 0.7843,  ..., 0.7922, 0.7843, 0.7804],
           [0.7922, 0.8314, 0.5451,  ..., 0.7882, 0.7804, 0.7765]]],
 
 
         [[[0.6745, 0.6784, 0.6824,  ..., 0.6745, 0.6706, 0.6667],
           [0.6824, 0.6863, 0.6902,  ..., 0.6863, 0.6784, 0.6784],
           [0.6941, 0.6980, 0.7020,  ..., 0.7020, 0.6941, 0.6863],
           ...,
           [0.7686, 0.7804, 0.7882,  ..., 0.8314, 0.8235, 0.8157],
           [0.4902, 0.4980, 0.5020,  ..., 0.7804, 0.7686, 0.7882],
           [0.4078, 0.4118, 0.4196,  ..., 0.5725, 0.6353, 0.6706]]],
 
 
         [[[0.7137, 0.7059, 0.7059,  ..., 0.6314, 0.6275, 0.6196],
           [0.7176, 0.7137, 0.7137,  ..., 0.6392, 0.6314, 0.6275],
           [0.7255

In [27]:
batch[0].shape

torch.Size([32, 1, 28, 28])

In [28]:
batch[1].shape

torch.Size([32])

## 1.2 Creating a Convolutional Model

In [58]:
n_classes = 24
kernel_size = 3
flattened_img_size = 75 * 3 * 3

model = nn.Sequential(
    # First convolution
    nn.Conv2d(IMG_CHS, 25, kernel_size, stride=1, padding=1),  # 25 x 28 x 28
    nn.BatchNorm2d(25),
    nn.ReLU(),
    nn.MaxPool2d(2, stride=2),  # 25 x 14 x 14
    # Second convolution
    nn.Conv2d(25, 50, kernel_size, stride=1, padding=1),  # 50 x 14 x 14
    nn.BatchNorm2d(50),
    nn.ReLU(),
    nn.Dropout(.2),
    nn.MaxPool2d(2, stride=2),  # 50 x 7 x 7
    # Third convolution
    nn.Conv2d(50, 75, kernel_size, stride=1, padding=1),  # 75 x 7 x 7
    nn.BatchNorm2d(75),
    nn.ReLU(),
    nn.MaxPool2d(2, stride=2),  # 75 x 3 x 3
    # Flatten to Dense
    nn.Flatten(),
    nn.Linear(flattened_img_size, 512),
    nn.Dropout(.3),
    nn.ReLU(),
    nn.Linear(512, n_classes)
)

## 1.3 The Model

In [59]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

False

In [60]:
compiled_model = torch.compile(model.to(device))
compiled_model

OptimizedModule(
  (_orig_mod): Sequential(
    (0): Conv2d(1, 25, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(25, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(50, 75, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): BatchNorm2d(75, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (13): Flatten(start_dim=1, end_dim=-1)
    (14): Linear(in_features=675, out_features=512, bias=True)
    (15): Dropout

In [63]:
loss_function = nn.CrossEntropyLoss()
optimizer = Adam(compiled_model.parameters())

In [64]:
def get_batch_accuracy(output, y, N):
    pred = output.argmax(dim=1, keepdim=True)
    correct = pred.eq(y.view_as(pred)).sum().item()
    return correct / N

### 1.3.1 Training the Model

In [65]:
def validate():
    loss = 0
    accuracy = 0

    model.eval()
    with torch.no_grad():
        for x, y in valid_loader:
            output = model(x)

            loss += loss_function(output, y).item()
            accuracy += get_batch_accuracy(output, y, valid_N)
    print('Valid - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

In [66]:
def train():
    loss = 0
    accuracy = 0

    model.train()
    for x, y in train_loader:
        output = model(x)
        optimizer.zero_grad()
        batch_loss = loss_function(output, y)
        batch_loss.backward()
        optimizer.step()

        loss += batch_loss.item()
        accuracy += get_batch_accuracy(output, y, train_N)
    print('Train - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

In [67]:
epochs = 20

for epoch in range(epochs):
    print('Epoch: {}'.format(epoch))
    train()
    validate()

Epoch: 0
Train - Loss: 242.9092 Accuracy: 0.9174
Valid - Loss: 30.5833 Accuracy: 0.9498
Epoch: 1
Train - Loss: 12.3297 Accuracy: 0.9962
Valid - Loss: 12.2767 Accuracy: 0.9813
Epoch: 2
Train - Loss: 14.6817 Accuracy: 0.9955
Valid - Loss: 32.6841 Accuracy: 0.9555
Epoch: 3
Train - Loss: 11.7286 Accuracy: 0.9959
Valid - Loss: 16.5277 Accuracy: 0.9720
Epoch: 4
Train - Loss: 0.7642 Accuracy: 0.9999
Valid - Loss: 12.3447 Accuracy: 0.9824
Epoch: 5
Train - Loss: 15.2806 Accuracy: 0.9956
Valid - Loss: 16.8242 Accuracy: 0.9816
Epoch: 6
Train - Loss: 3.0905 Accuracy: 0.9989
Valid - Loss: 10.0717 Accuracy: 0.9868
Epoch: 7
Train - Loss: 12.1556 Accuracy: 0.9956
Valid - Loss: 15.8152 Accuracy: 0.9782
Epoch: 8
Train - Loss: 1.0786 Accuracy: 0.9996
Valid - Loss: 10.6472 Accuracy: 0.9840
Epoch: 9
Train - Loss: 9.3197 Accuracy: 0.9969
Valid - Loss: 22.0103 Accuracy: 0.9657
Epoch: 10
Train - Loss: 5.0580 Accuracy: 0.9984
Valid - Loss: 15.7148 Accuracy: 0.9784
Epoch: 11
Train - Loss: 0.3224 Accuracy: 1.000

## 1.4 Cleaning the memory

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)